In [13]:
#read the vdip,ra,dec,z_l from zans info
#also slect a subsample, using skewness/sn...., create a folder to save spectra/zoomed spectra....
#write catalog with info, [plate,fiber,mjd,z_lens,z_src,wave_main,wave_sub, flux_total, einstein_radius, ra, dec, skewness, width, flux_main, flux_sub, tag, note]  
#mag_i !!!! still need this

In [2]:
import json 
import re
import os
import numpy as np
from astropy.table import Table
from astropy.io import ascii
import h5py
import sys
sys.path.append('../')
import utils as ul

#this block read the info, and do cutting
dir0 = os.getcwd()
t = Table.read('../../data/catalog_1_by_zoomed_spectra.csv', format='ascii.csv')
#index_all = t['id@json'].data
plate_all = t['plate'].data
mjd_all = t['mjd'].data
fiber_all = t['fiber'].data
tag_all = t['tag'].data
note_all = t['note'].data
zs_all = t['main_wave'].data/1215.67-1.0

ra_all = []
dec_all = []
vdisp_all = []
zl_all = []
#read zans.ra,dec,vdisp info
base_dir = '../../data/2/'
for ii in range(len(plate_all)): #
    file_name = '{}_{}.h5'.format(plate_all[ii],mjd_all[ii])
    h5_file = base_dir+file_name
    with h5py.File(h5_file, 'r') as f:
        arr2 = f['zans']
        zans_arr = arr2[:]
    #print('file {} read finished'.format(file_name))
    fibers = np.array([item[3] for item in zans_arr],dtype=np.int)  #refer to below to find index, Note index-1 !!!
    id_fiber = np.where(fibers == int(fiber_all[ii]))
    
    #extract the structure which save the spectrum info
    struct = zans_arr[id_fiber][0]
    #save some info
    ra = struct[8]
    dec = struct[9]
    #print(ra,dec)\
    z_l = struct[12]
    vdisp = struct[22]

    ra_all.append(ra)
    dec_all.append(dec)
    vdisp_all.append(vdisp)
    zl_all.append(z_l)
    
    if ii%100 == 1:
        print('read {} file'.format(ii))

read 1 file
read 101 file
read 201 file
read 301 file
read 401 file
read 501 file
read 601 file


In [36]:
from astropy.table import Column
from copy import deepcopy
ra_col = Column(ra_all, name='ra')
dec_col = Column(dec_all, name='dec')
zl_col = Column(zl_all, name='z_l')
zs_col = Column(zs_all, name='z_s')
vdisp_col = Column(vdisp_all, name='vdisp') #km/s

cal_thetaE = np.vectorize(ul.cal_einstein_radius_in_arcsec)
thetaE_all = cal_thetaE(zl_col.data,zs_col.data,vdisp_col.data)
thetaE_col = Column(thetaE_all , name='thetaE')

tnew = deepcopy(t)
tnew.add_columns([ra_col,dec_col,zl_col,zs_col,vdisp_col,thetaE_col])
tnew.write('../../data/catalog_2_by_plot_einstien.csv',format='ascii.csv',overwrite=True)
tnew.write('./1.fits',format='fits',overwrite=True)

In [37]:
tnew.info

<Table length=690>
     name      dtype 
------------- -------
        plate   int64
          mjd   int64
        fiber   int64
          tag    str6
         note   str35
 sn_peak_wave float64
      peak_sn float64
      spec_id   int64
      peak_id   int64
if_multi_line    str5
    main_wave float64
main_skewness float64
    main_flux float64
   main_width float64
     sub_wave float64
 sub_skewness float64
     sub_flux float64
    sub_width float64
     tot_flux float64
           ra float64
          dec float64
          z_l float32
          z_s float64
        vdisp float32
       thetaE float64

In [12]:
print(ra_all[0])
print(dec_all[0])
print(zl_all[0])
print(vdisp_all[0])

138.32591000000002
33.992633
0.7646578
330.20834


In [32]:
from astropy.io import fits
all_data_catalog = fits.getdata('/data/inspur_disk03/userdir/caoxy/sdss_data/spAll-v5_13_0.fits',memmap=True)
#all_data_catalog.names.index('FIBERID')
cmodel_mag_spall = all_data_catalog['CMODELMAG'][:,3]
model_mag_spall = all_data_catalog['MODELMAG'][:,3]

plate_spall = all_data_catalog['PLATE']
mjd_spall = all_data_catalog['MJD']
fiber_spall = all_data_catalog['FIBERID']

In [33]:
cmodel_mag_all = []
model_mag_all = []
for ii,item in enumerate(plate_all[:]):
    condition = np.logical_and((plate_spall==plate_all[ii]), (mjd_spall==mjd_all[ii]))
    condition = np.logical_and(condition, (fiber_spall==fiber_all[ii]))
    id_pickle = np.where(condition)[0][0]
    cmodel_mag_all.append(cmodel_mag_spall[id_pickle])
    model_mag_all.append(model_mag_spall[id_pickle])

In [40]:
cmodel_mag_col = Column(cmodel_mag_all, name='cmodel_mag_i')
model_mag_col = Column(model_mag_all, name='model_mag_i')

tnew2 = deepcopy(tnew)
tnew2.add_columns([cmodel_mag_col,model_mag_col])
tnew2.write('../../data/catalog_3_by_plot_einstien.csv',format='ascii.csv',overwrite=True)
tnew2.write('./2.fits',format='fits',overwrite=True)